In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  G_path = '/content/drive/MyDrive/DataSources/Project_Data'
  !pip install pickle5
except:
  G_path = '/content/drive/MyDrive/DataSources/Project_Data'

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 5.0 MB/s 


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset as dset
#import torchvision.transforms.Compose
import numpy as np
from datetime import datetime	
import random
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import random_split
from torch.utils.data import SubsetRandomSampler
from torch.utils.data import WeightedRandomSampler
from torch.utils.data import DataLoader
import time
from tqdm import tqdm
import pickle as pickle
from statistics import mean
import pandas as pd
pd.options.display.max_colwidth = 250
import sklearn as sk
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else "cpu")
from sklearn.utils import resample
import warnings
warnings.filterwarnings('ignore')

In [17]:
Binary_classification = True
file = 'Input_DataSet_280x210' 
input_data , labels = pickle.load(open(G_path + '/06_Rescaled_DataSet/'+ file,'rb'))
file = 'Balanced_Test_Set_280x210' 
Evaluation_set = pickle.load(open(G_path + '/06_Rescaled_DataSet/' + file,'rb'))

if Binary_classification:
  labels = labels[1]
else:
  labels = labels[0]

In [18]:
labels[1][:10]

tensor([0, 0, 1, 0, 0, 1, 0, 0, 0, 0])

# **CNN Networks**

In [5]:
class CNN_Nets:
  class Net8(nn.Module):
    def __init__(self,drop_out):
        super().__init__()
        self.conv1 = nn.Conv2d( in_channels=3 , out_channels=64, stride = 2 , kernel_size=(3, 3))
        self.pool1 = nn.MaxPool2d( kernel_size = (3,3), stride = 2, padding = 0 )
        self.conv2 = nn.Conv2d( in_channels=64, out_channels=128, stride = 2 , kernel_size=(3, 3))
        self.pool2 = nn.MaxPool2d( kernel_size = (3,3), stride = 2, padding = 0 )
        self.conv3 = nn.Conv2d( in_channels=128, out_channels=64, stride = 2 , kernel_size=(3, 3))
        self.pool3 = nn.MaxPool2d( kernel_size = (3,3), stride = 2, padding = 0 )
        self.fc1   = nn.Linear(in_features= 384 , out_features = 128)
        self.fc2   = nn.Linear(in_features= 128 , out_features = 7)
        self.Act   = nn.LeakyReLU(inplace=True)
        self.dropout = nn.Dropout(drop_out)
    def forward(self, x):
        out = self.Act(self.conv1(x))
        out = self.Act(self.pool1(out))
        out = self.dropout(out)
        out = self.Act(self.conv2(out))
        out = self.Act(self.pool2(out))
        out = self.dropout(out)
        out = self.Act(self.conv3(out))
        out = self.Act(self.pool3(out))
        out = self.dropout(out)
        out = torch.flatten(out, 1) 
        out = self.Act(self.fc1(out))
        out = self.fc2(out)
        return out
    def forward_noDrop(self, x):
        out = self.Act(self.conv1(x))
        out = self.Act(self.pool1(out))
        out = self.Act(self.conv2(out))
        out = self.Act(self.pool2(out))
        out = self.Act(self.conv3(out))
        out = self.Act(self.pool3(out))
        out = torch.flatten(out, 1) 
        out = self.Act(self.fc1(out))
        out = self.fc2(out)
        return out

  class Net8_a(nn.Module):
    def __init__(self,drop_out):
        super().__init__()
        self.conv1 = nn.Conv2d( in_channels=3 , out_channels=64, stride = 2 , kernel_size=(3, 3))
        self.pool1 = nn.MaxPool2d( kernel_size = (3,3), stride = 2, padding = 0 )
        self.conv2 = nn.Conv2d( in_channels=64, out_channels=128, stride = 2 , kernel_size=(3, 3))
        self.pool2 = nn.MaxPool2d( kernel_size = (3,3), stride = 2, padding = 0 )
        self.conv3 = nn.Conv2d( in_channels=128, out_channels=256, stride = 2 , kernel_size=(3, 3))
        self.pool3 = nn.MaxPool2d( kernel_size = (3,3), stride = 2, padding = 0 )
        self.fc1   = nn.Linear(in_features= 1536 , out_features = 128)
        self.fc2   = nn.Linear(in_features= 128 , out_features = 7)
        self.Act   = nn.LeakyReLU()
        self.dropout = nn.Dropout(drop_out)
    def forward(self, x):
        out = self.Act(self.conv1(x))
        out = self.pool1(out)
        out = self.dropout(out)
        out = self.Act(self.conv2(out))
        out = self.pool2(out)
        out = self.dropout(out)
        out = self.Act(self.conv3(out))
        out = self.pool3(out)
        out = self.dropout(out)
        out = torch.flatten(out, 1) 
        out = self.Act(self.fc1(out))
        out = self.fc2(out)
        return out
    def forward_noDrop(self, x):
        out = self.Act(self.conv1(x))
        out = self.pool1(out)
        out = self.Act(self.conv2(out))
        out = self.pool2(out)
        out = self.Act(self.conv3(out))
        out = self.pool3(out)
        out = torch.flatten(out, 1) 
        out = self.Act(self.fc1(out))
        out = self.fc2(out)
        return out
  
  class Net8_b(nn.Module):
    def __init__(self,drop_out):
        super().__init__()
        self.conv1 = nn.Conv2d( in_channels=3 , out_channels=64, stride = 2 , kernel_size=(3, 3))
        self.pool1 = nn.MaxPool2d( kernel_size = (3,3), stride = 2, padding = 0 )
        self.conv2 = nn.Conv2d( in_channels=64, out_channels=128, stride = 2 , kernel_size=(3, 3))
        self.pool2 = nn.MaxPool2d( kernel_size = (3,3), stride = 2, padding = 0 )
        self.conv3 = nn.Conv2d( in_channels=128, out_channels=256, stride = 2 , kernel_size=(3, 3))
        self.pool3 = nn.MaxPool2d( kernel_size = (3,3), stride = 2, padding = 0 )
        self.fc1   = nn.Linear(in_features= 1536 , out_features = 128)
        self.fc2   = nn.Linear(in_features= 128 , out_features = 7)
        self.Act   = nn.LeakyReLU(inplace=True)
        self.dropout = nn.Dropout(drop_out)
    def forward(self, x):
        out = self.Act(self.conv1(x))
        out = self.Act(self.pool1(out))
        out = self.dropout(out)
        out = self.Act(self.conv2(out))
        out = self.Act(self.pool2(out))
        out = self.dropout(out)
        out = self.Act(self.conv3(out))
        out = self.Act(self.pool3(out))
        out = self.dropout(out)
        out = torch.flatten(out, 1) 
        out = self.Act(self.fc1(out))
        out = self.fc2(out)
        return out
    def forward_noDrop(self, x):
        out = self.Act(self.conv1(x))
        out = self.Act(self.pool1(out))
        out = self.Act(self.conv2(out))
        out = self.Act(self.pool2(out))
        out = self.Act(self.conv3(out))
        out = self.Act(self.pool3(out))
        out = torch.flatten(out, 1) 
        out = self.Act(self.fc1(out))
        out = self.fc2(out)
        return out


  class Net8_b_binary(nn.Module):
    def __init__(self,drop_out):
        super().__init__()
        self.conv1 = nn.Conv2d( in_channels=3 , out_channels=64, stride = 2 , kernel_size=(3, 3))
        self.pool1 = nn.MaxPool2d( kernel_size = (3,3), stride = 2, padding = 0 )
        self.conv2 = nn.Conv2d( in_channels=64, out_channels=128, stride = 2 , kernel_size=(3, 3))
        self.pool2 = nn.MaxPool2d( kernel_size = (3,3), stride = 2, padding = 0 )
        self.conv3 = nn.Conv2d( in_channels=128, out_channels=256, stride = 2 , kernel_size=(3, 3))
        self.pool3 = nn.MaxPool2d( kernel_size = (3,3), stride = 2, padding = 0 )
        self.fc1   = nn.Linear(in_features= 1536 , out_features = 128)
        self.fc2   = nn.Linear(in_features= 128 , out_features = 2)
        self.Act   = nn.LeakyReLU(inplace=True)
        self.dropout = nn.Dropout(drop_out)
    def forward(self, x):
        out = self.Act(self.conv1(x))
        out = self.Act(self.pool1(out))
        out = self.dropout(out)
        out = self.Act(self.conv2(out))
        out = self.Act(self.pool2(out))
        out = self.dropout(out)
        out = self.Act(self.conv3(out))
        out = self.Act(self.pool3(out))
        out = self.dropout(out)
        out = torch.flatten(out, 1) 
        out = self.Act(self.fc1(out))
        out = self.fc2(out)
        return out
    def forward_noDrop(self, x):
        out = self.Act(self.conv1(x))
        out = self.Act(self.pool1(out))
        out = self.Act(self.conv2(out))
        out = self.Act(self.pool2(out))
        out = self.Act(self.conv3(out))
        out = self.Act(self.pool3(out))
        out = torch.flatten(out, 1) 
        out = self.Act(self.fc1(out))
        out = self.fc2(out)
        return out

# **Test Nets**

In [6]:
model_test = CNN_Nets.Net8_a(drop_out=0.1)
X_ = input_data[0][0:5]
out = model_test(X_)
out.shape

torch.Size([5, 7])

# **Metrics and Performance**

In [8]:
def confusion_matrix(Y,Y_pred):
    CM = sk.metrics.confusion_matrix(Y,Y_pred)
    print('Nr. of Data : \n', CM.sum())
    print('Accuracy of The Model : \n', np.diag(CM).sum()/CM.sum())
    sk.metrics.ConfusionMatrixDisplay(CM).plot()
    FP = CM.sum(axis=0) - np.diag(CM) 
    FN = CM.sum(axis=1) - np.diag(CM)
    TP = np.diag(CM)
    TN = CM.sum() - (FP + FN + TP)
    weights = CM.sum(axis=1) / CM.sum() 
    ACC = np.nan_to_num((TP+TN)/(TP+FP+FN+TN) , nan=0)
    Recall_Sensitivity = np.nan_to_num(TP/(TP+FN) , nan=0)
    Specificity = np.nan_to_num(TN/(TN+FP) , nan=0)
    Precision = np.nan_to_num(TP/(TP+FP) , nan=0)
    f1_score = np.nan_to_num( 2*Precision*Recall_Sensitivity / (Recall_Sensitivity + Precision), nan=0)
    Performance_DF = pd.concat([pd.DataFrame(CM),pd.DataFrame(weights, columns=['weights']),pd.DataFrame(Precision, columns=['Precision']),pd.DataFrame(Recall_Sensitivity,columns=['Recall_Sensitivity'])
        ,pd.DataFrame(Specificity, columns=['Specificity']),pd.DataFrame(f1_score, columns=['f1_score'])], axis=1)
    total_row1 = pd.Series({'Precision':mean(Precision),'Recall_Sensitivity':mean(Recall_Sensitivity),'Specificity':mean(Specificity),'f1_score':mean(f1_score)}, name='Simple Avg.')
    total_row2 = pd.Series({'Precision':sum(weights*Precision),'Recall_Sensitivity':sum(weights*Recall_Sensitivity),'Specificity':sum(weights*Specificity),'f1_score':sum(weights*f1_score)}, name='Weighted Avg.')
    Performance_DF = Performance_DF.append([total_row1,total_row2])
    cols = ['weights','Precision','Recall_Sensitivity','Specificity','f1_score']
    per_details = Performance_DF[cols].style.format({'weights': "{:.1%}",'Precision': "{:.1%}",'Recall_Sensitivity': "{:.1%}",'Specificity': "{:.1%}",'f1_score': "{:.1%}"})
    return per_details

def recall_specificity(Y,Y_pred, type):
    CM = sk.metrics.confusion_matrix(Y,Y_pred)
    FP = CM.sum(axis=0) - np.diag(CM) 
    FN = CM.sum(axis=1) - np.diag(CM)
    TP = np.diag(CM)
    TN = CM.sum() - (FP + FN + TP)
    weights = CM.sum(axis=1) / CM.sum() 
    ACC = np.nan_to_num((TP+TN)/(TP+FP+FN+TN) , nan=0)
    Recall_Sensitivity = np.nan_to_num(TP/(TP+FN) , nan=0)
    Specificity = np.nan_to_num(TN/(TN+FP) , nan=0)
    Precision = np.nan_to_num(TP/(TP+FP) , nan=0)
    f1_score = np.nan_to_num( 2*Precision*Recall_Sensitivity / (Recall_Sensitivity + Precision), nan=0)
    Performance_DF = pd.concat([pd.DataFrame(CM),pd.DataFrame(weights, columns=['weights']),pd.DataFrame(Precision, columns=['Precision']),pd.DataFrame(Recall_Sensitivity,columns=['Recall_Sensitivity'])
        ,pd.DataFrame(Specificity, columns=['Specificity']),pd.DataFrame(f1_score, columns=['f1_score'])], axis=1)
    total_row1 = pd.Series({'Precision':mean(Precision),'Recall_Sensitivity':mean(Recall_Sensitivity),'Specificity':mean(Specificity),'f1_score':mean(f1_score)}, name='Simple Avg.')
    total_row2 = pd.Series({'Precision':sum(weights*Precision),'Recall_Sensitivity':sum(weights*Recall_Sensitivity),'Specificity':sum(weights*Specificity),'f1_score':sum(weights*f1_score)}, name='Weighted Avg.')
    Performance_DF = Performance_DF.append([total_row1,total_row2])
    cols = ['weights','Precision','Recall_Sensitivity','Specificity','f1_score']
    return per_details

def plot_loss_accuracy(model_):
    epochs_X = [i for i in range(1, len(model_.Epochs_Train_loss)+1)]
    fig, axs = plt.subplots(1,2,figsize=(14,4))
    axs[0].plot(epochs_X , model_.Epochs_Train_loss , 'bo-', label='Train loss')
    axs[0].plot(epochs_X , model_.Epochs_Val_loss,'ro-', label='Validation loss')
    axs[0].plot(epochs_X , model_.Epochs_test_loss,'go-', label='Test loss')
    axs[0].set_xlabel("Epochs", fontsize = 12)
    axs[0].set_ylabel("Loss", fontsize = 12)
    axs[0].grid()
    axs[0].legend()
    axs[0].set_title('Train and Validation loss by epochs', fontsize = 14)
    axs[1].plot(epochs_X , model_.Epochs_Train_Acc , 'bo-', label='Train Accuracy')
    axs[1].plot(epochs_X , model_.Epochs_Val_Acc ,'ro-', label='Validation Accuracy')
    axs[1].plot(epochs_X , model_.Epochs_test_Acc ,'go-', label='Test Accuracy')
    axs[1].set_xlabel("Epochs", fontsize = 12)
    axs[1].set_ylabel("Accuracy", fontsize = 12)
    axs[1].grid()
    axs[1].legend()
    axs[1].set_title('Train and Validation Accuracy by epochs', fontsize = 14)
    plt.show()

def recall_specificity_precision(Y,Y_pred, weighted_avg):
    CM = sk.metrics.confusion_matrix(Y,Y_pred)
    FP = CM.sum(axis=0) - np.diag(CM) 
    FN = CM.sum(axis=1) - np.diag(CM)
    TP = np.diag(CM)
    TN = CM.sum() - (FP + FN + TP)
    weights = CM.sum(axis=1) / CM.sum() 
    Accuracy = np.nan_to_num((TP+TN)/(TP+FP+FN+TN) , nan=0)
    Recall = np.nan_to_num(TP/(TP+FN) , nan=0)
    Specificity = np.nan_to_num(TN/(TN+FP) , nan=0)
    Precision = np.nan_to_num(TP/(TP+FP) , nan=0)
    if weighted_avg:
        return round(sum(weights*Recall),3), round(sum(weights*Specificity),3), round(sum(weights*Precision),3)
    else:
        return round(mean(Recall),3), round(mean(Specificity),3), round(mean(Precision),3)

### **1.3 Training**

In [9]:
class Model_Training_with_loader:

    def __init__(self, Net, Drop, LR, batch_size , Momentum, epochs,patience, weight_decay, loss_func, opt_func,w_sampler, trainDataset, validDataset, X_test,Y_test, print_epochs,hyper_params):    
        
        self.model = Net(Drop).to(device)
        if opt_func is torch.optim.Adam:
            self.opt = opt_func(self.model.parameters(), lr=LR, weight_decay=weight_decay)
        else:
            self.opt = opt_func(self.model.parameters(), lr=LR,momentum=Momentum, weight_decay=weight_decay)

        self.loss_func = loss_func()
        self.epochs = epochs
        self.patience = patience
        self.print_epochs = print_epochs
        self.batch_size = batch_size
        self.Epochs_Train_loss = []
        self.Epochs_Train_Acc = []
        self.Epochs_Val_loss = []
        self.Epochs_Val_Acc = []
        self.Epochs_test_loss = []
        self.Epochs_test_Acc = []
        self.hyper_params = hyper_params
        self.Y_test = Y_test
        self.X_test = X_test
        self.train_loader = DataLoader(dataset = trainDataset , sampler = w_sampler, batch_size = self.batch_size, num_workers=4)
        self.valid_loader = DataLoader(dataset = validDataset , shuffle=True, batch_size = self.batch_size, num_workers=2)
        self.test_loader = DataLoader(dataset = testDataset , shuffle=True, batch_size = self.batch_size, num_workers=2)

    def train(self):
        
        model = self.model
        loss_fn = self.loss_func
        opt = self.opt 
        batch_size = self.batch_size
        min_loss = 100
        iters = 0

        for epoch in range(1, self.epochs+1 ):
            start_time=time.time()
            steps_train_loss = []
            steps_train_Acc = []
            steps_val_loss = []
            steps_val_Acc = []
            steps_test_loss = []
            steps_test_Acc = []
            torch.cuda.empty_cache()
            for batch, (X, Y) in enumerate(self.train_loader):
                X = X.to(device)
                Y = Y.to(device)
                opt.zero_grad()
                model.train()
                y_pred = model.forward(X)
                loss = loss_fn(y_pred, Y)
                loss.backward()
                opt.step()
                y_pred = y_pred.argmax(axis=1)
                nr_of_corrects = (y_pred == Y).sum().item()
                step_acc = nr_of_corrects / batch_size
                steps_train_Acc.append(step_acc)
                steps_train_loss.append(loss.item())
                
                # if (i+1) % 200 == 0:    # print every 2000 mini-batches
                #     print('[Epoch: {}, Nr. Batch: {}]  , Train-Steps-loss: {:.1f} , running_acc: {:.1%}'.format(epoch , i+1 , train_steps_loss , batch_nr_correct / train_nr_total))
                #     self.train_steps_acc = []
                #     train_steps_loss = 0

              #validation loss calculation
            
            for batch, (X, Y) in enumerate(self.valid_loader):
                X = X.to(device)
                Y = Y.to(device)
                model.eval()
                Y_pred = model(X)
                loss_ = loss_fn(Y_pred, Y)
                epoch_loss = loss_.item()
                Y_pred = Y_pred.argmax(axis=1)
                nr_correct = (Y_pred == Y).sum().item()
                step_acc = nr_correct / batch_size
                steps_val_Acc.append(step_acc)
                steps_val_loss.append(epoch_loss)
                
            #Test Set Performance
            for batch, (X, Y) in enumerate(self.test_loader):
                X = X.to(device)
                Y = Y.to(device)
                model.eval()
                Y_pred = model(X)
                loss_ = loss_fn(Y_pred, Y)
                epoch_loss = loss_.item()
                Y_pred = Y_pred.argmax(axis=1)
                nr_correct = (Y_pred == Y).sum().item()
                step_acc = nr_correct / batch_size
                steps_test_Acc.append(step_acc)
                steps_test_loss.append(epoch_loss)

            # Epoch Performance Metrics
            train_epoch_loss = mean(steps_train_loss)
            train_epoch_Acc = mean(steps_train_Acc)
            self.Epochs_Train_loss.append(train_epoch_loss)
            self.Epochs_Train_Acc.append(train_epoch_Acc)   
            val_epoch_loss = mean(steps_val_loss)
            val_epoch_Acc = mean(steps_val_Acc)
            self.Epochs_Val_loss.append(val_epoch_loss)
            self.Epochs_Val_Acc.append(val_epoch_Acc)
            test_epoch_loss = mean(steps_test_loss)
            test_epoch_Acc = mean(steps_test_Acc)           
            self.Epochs_test_loss.append(test_epoch_loss)
            self.Epochs_test_Acc.append(test_epoch_Acc)
            End_time = time.time() 

            if val_epoch_loss < min_loss:
              min_loss = val_epoch_loss
              pickle.dump(model,open('Best_Model','wb'))
              iters = 0
            else:
              iters +=1
     
            if iters > self.patience:
              model = pickle.load(open('Best_Model','rb'))
              print(f'Earlt Stoppo happen at Epoche {epoch} after no improvment of {iters} epochs ')
              break

            if self.print_epochs:
                print(f'[Epoch: {epoch}]  , Train_loss: {train_epoch_loss:.1f} , Train_Acc: {train_epoch_Acc:.1%}, Val_loss: {val_epoch_loss:.1f} , Val_Acc: {val_epoch_Acc:.1%}, Test_Acc: {test_epoch_Acc:.1%}  , run time: {np.round(End_time - start_time, 2)}')
        # print('Finished Training')

# **Define Training data**

In [11]:
X_train = input_data[0]
Y_train = labels[0]
X_valid = input_data[1]
Y_valid = labels[1]
X_test = input_data[2]
Y_test = labels[2]


label_freq = np.bincount(Y_train)
labels_weights = 1. / label_freq
weights = labels_weights[Y_train]
w_sampler = WeightedRandomSampler(weights, len(weights))

trainDataset = dset(X_train, Y_train)
validDataset = dset(X_valid, Y_valid)
testDataset = dset(X_test, Y_test)

# **Model by Grid**

In [14]:

Net = [CNN_Nets.Net8_b_binary]
Drop = [0.2,0.25]
LR = [ 1.1e-3,1e-2,1e-4]
batch_size = [24,32]
Momentum = [0.77,.85]
epochs = [100]
patience = [20]
weight_decay = [1e-3,1e-5]
loss_func  =  [nn.CrossEntropyLoss]
opt_func = [torch.optim.SGD]


grid = {
    'Net' : Net
    ,'Drop' : Drop
    ,'LR' : LR
    ,'batch_size' : batch_size
    ,'Momentum' : Momentum
    ,'epochs' : epochs
    ,'patience': patience
    ,'weight_decay' :weight_decay
    ,'loss_func'  :loss_func
    ,'opt_func' : opt_func
}
params = sk.model_selection.ParameterGrid(grid)

In [15]:
print_epochs = False
export_csv = True
save_models = True
export_name = 'CNN_280x210_Binary' # {'280_210_','AutoEncoder_'}

Hyper_Details = pd.DataFrame(columns=['model_name','hyper_param','test_overall_metric','test_recall','blc_test_recall','valid_recall','train_accuracy','valid_accuracy','test_accuracy','train_epoch_loss','train_epoch_acc','valid_epoch_loss','valid_epoch_acc','test_epoch_acc','test_epoch_loss'])
date_hour = datetime.now().strftime("%d_%b%y_%H-%M") 
Max_train_Acc = 0  
Max_valid_Acc = 0 
Max_test_Acc = 0                  
Max_test_Recall = 0
Max_BLC_Test_Recall = 0
Max_test_overall_metric = 0
i = -1
pbar = tqdm(params)
for p in pbar:
    i += 1
    model_name =  export_name + 'Model' + str(i) + '_' + date_hour
    torch.cuda.empty_cache()
    Model_ = Model_Training_with_loader(**p,w_sampler = w_sampler , trainDataset = trainDataset, validDataset = validDataset , X_test = X_test, Y_test = Y_test, print_epochs =print_epochs,hyper_params=p)
    np.random.seed(0)
    random.seed(0)
    torch.manual_seed(0)
    Model_.train()

    train_accuracy = max(Model_.Epochs_Train_Acc)
    # valid_accuracy = max(Model_.Epochs_Val_Acc)
    # test_accuracy = max(Model_.Epochs_test_Acc)
    train_epoch_loss = Model_.Epochs_Train_loss
    train_epoch_acc = Model_.Epochs_Train_Acc
    valid_epoch_loss = Model_.Epochs_Val_loss
    valid_epoch_acc = Model_.Epochs_Val_Acc
    test_epoch_loss = Model_.Epochs_test_loss
    test_epoch_acc = Model_.Epochs_test_Acc


    model_ = Model_.model.eval().to('cpu')
    # test Avg. Sensitivity
    X = input_data[2]
    Y = labels[2]
    Y_pred = model_.forward_noDrop(X).argmax(dim=1)
    test_accuracy = sk.metrics.accuracy_score(Y, Y_pred )
    test_precision, test_recall, test_fscore, m = sk.metrics.precision_recall_fscore_support(Y, Y_pred, average = 'macro')
    # Valid Avg. Sensitivity
    X = input_data[1]
    Y = labels[1]
    Y_pred = model_.forward_noDrop(X).argmax(dim=1)
    valid_accuracy = sk.metrics.accuracy_score(Y, Y_pred )
    valid_precision, valid_recall, valid_fscore, m = sk.metrics.precision_recall_fscore_support(Y, Y_pred, average = 'macro')
    # Balanced test set Avg. Sensitivity
    X = Evaluation_set[0]
    Y = Evaluation_set[1]
    Y_pred = model_.forward_noDrop(X).argmax(dim=1)
    blc_test_precision, blc_test_recall, blc_test_fscore, m = sk.metrics.precision_recall_fscore_support(Y, Y_pred, average = 'macro')

    test_overall_metric = (test_accuracy + test_recall + blc_test_recall ) / 3
    if test_overall_metric > Max_test_overall_metric: pickle.dump(model_ , open(G_path +'/08_Saved_Models_Outpus/Models/CNN_Grid_Search_Models/' +  'Best_Grid_Model' +str(i) +'_' + date_hour, 'wb'))

    if train_accuracy > Max_train_Acc: Max_train_Acc = train_accuracy
    if valid_accuracy > Max_valid_Acc: Max_valid_Acc = valid_accuracy
    if test_accuracy > Max_test_Acc: Max_test_Acc = test_accuracy
    if test_recall > Max_test_Recall: Max_test_Recall = test_recall
    if blc_test_recall > Max_BLC_Test_Recall: Max_BLC_Test_Recall = blc_test_recall

    pbar.set_description(f'Max_Test_Recall: {Max_test_Recall:.1%}, Max_BLC_Test_Recall: {Max_BLC_Test_Recall:.1%}, Max_Tr_Acc: {Max_train_Acc:.1%}, Max_V_Acc: {Max_valid_Acc:.1%}, Max_Te_Acc: {Max_test_Acc:.1%} ')
    pbar.set_postfix_str(f'Train_Acc: {train_accuracy:.1%} , Valide_Acc: {valid_accuracy:.1%} , Test_Acc: {test_accuracy:.1%}')

    new_row = pd.Series({'model_name':model_name,'hyper_param':p,'test_overall_metric':test_overall_metric,'test_recall':test_recall,'blc_test_recall':blc_test_recall,'valid_recall':valid_recall,'train_accuracy':train_accuracy,'valid_accuracy':valid_accuracy,'test_accuracy':test_accuracy,
                         'train_epoch_loss':train_epoch_loss,'train_epoch_acc':train_epoch_acc,'valid_epoch_loss':valid_epoch_loss,'valid_epoch_acc':valid_epoch_acc,'test_epoch_acc':test_epoch_acc,'test_epoch_loss':test_epoch_loss}, name='')
    Hyper_Details = Hyper_Details.append(new_row)
    m_details = pd.DataFrame()
    m_details = m_details.append(new_row)




    if export_csv:
      # Hyper_Details.to_csv(G_path +'/Saved/Grid_Search_Results/' + export_name + date_hour+ '.csv')
      
      pickle.dump(m_details, open(G_path +'/08_Saved_Models_Outpus/Grid_Search_Results/' + model_name, 'wb'))
      pickle.dump(Hyper_Details, open(G_path +'/08_Saved_Models_Outpus/Grid_Search_Results/' + 'Grid_All_' + export_name + date_hour , 'wb'))

    if save_models:
      pickle.dump(model_, open(G_path +'/08_Saved_Models_Outpus/Models/CNN_Grid_Search_Models/' +  model_name, 'wb'))

    Best_Grid_Model = pickle.load(open(G_path +'/08_Saved_Models_Outpus/Models/CNN_Grid_Search_Models/' +  'Best_Grid_Model' +str(i) +'_' + date_hour, 'rb'))

Hyper_Details.sort_values('test_overall_metric', ascending=False,inplace=True)
best_param = Hyper_Details['hyper_param'][0]
best_params = Hyper_Details['hyper_param'][:4]

# Grid_Details = Hyper_Details

# Model_Grid = Model_Training_with_loader(**best_param,w_sampler = w_sampler , trainDataset = trainDataset, validDataset = validDataset , X_test = X_test, Y_test = Y_test, print_epochs =print_epochs,hyper_params=best_param)
# np.random.seed(0)
# random.seed(0)
# torch.manual_seed(0)
# Model_Grid.train()
# print('Max Avg. Recall on Test ====>> :')
# Grid_Details['test_recall'][:5], Grid_Details['hyper_param'][0]

Max_Test_Recall: 27.6%, Max_BLC_Test_Recall: 29.5%, Max_Tr_Acc: 21.5%, Max_V_Acc: 52.1%, Max_Te_Acc: 49.4% : 100%|██████████| 4/4 [01:10<00:00, 17.51s/it, Train_Acc: 20.0% , Valide_Acc: 46.9% , Test_Acc: 42.5%]


In [ ]:
# model_ = Model_Grid
model_1 = Best_Grid_Model
X = input_data[2]
Y = labels[2]
# X = Evaluation_set[0]
# Y = Evaluation_set[1]
Y_pred = model_1.forward_noDrop(X).argmax(dim=1)
results = confusion_matrix(Y,Y_pred)
# plot_loss_accuracy(model_)
results

# **Cross Validation: Repeat 5 times with 10 Kfold (overall 50 models)**

In [ ]:
# K_Folds_test = sk.model_selection.KFold(n_splits=10, shuffle=True)
K_Folds_valid = sk.model_selection.KFold(n_splits=10, shuffle=True)
dat_hour = datetime.now().strftime("%d_%b_%Y_%H_%M")
print_epochs = False

param1 = {
    'Net' : CNN_Nets.Net8_b_binary
    ,'Drop' : 0.2
    ,'LR' : 1.3e-3
    ,'batch_size' : 20
    ,'Momentum' : 0.85
    ,'epochs' : 100
    ,'patience': 20
    ,'weight_decay' :1e-3
    ,'loss_func'  :torch.nn.modules.loss.CrossEntropyLoss
    ,'opt_func' : torch.optim.SGD
}

CV_data = torch.cat([input_data[0],input_data[1]],dim=0)
CV_label = torch.cat([labels[0],labels[1]],dim=0)
X_test = input_data[2]
Y_test = labels[2]


CV_Details = pd.DataFrame(columns=['hyper_param','train_recall_weighed','valid_recall_weighed','test_recall_weighed'
                                  ,'valid_recall_simple','test_recall_simple'
                                  ,'valid_specificity_weighed','test_specificity_weighed'
                                  ,'valid_specificity_simple','test_specificity_simple'])

Max_train_Acc = 0  
Max_valid_Acc = 0 
Max_test_Acc = 0                  

pbar1 = tqdm(range(5))
for i in pbar1:
    pbar2 = tqdm(enumerate(K_Folds_valid.split(CV_data)), total=K_Folds_valid.get_n_splits())
    for fold, (training_index, valid_index) in pbar2:
        X_train = CV_data[training_index]
        Y_train = CV_label[training_index]
        X_valid = CV_data[valid_index]
        Y_valid = CV_label[valid_index]

        label_freq = np.bincount(Y_train)
        labels_weights = 1. / label_freq
        weights = labels_weights[Y_train]
        w_sampler = WeightedRandomSampler(weights, len(weights))

        trainDataset = dset(X_train, Y_train)
        validDataset = dset(X_valid, Y_valid)
        testDataset = dset(X_test, Y_test)

        Model_ = Model_Training_with_loader(**param1,w_sampler = w_sampler , trainDataset = trainDataset, validDataset = validDataset , X_test = X_test, Y_test = Y_test, print_epochs =print_epochs,hyper_params=param1)
        Model_.train()

        model_ = Model_.model.eval().to('cpu')
            
        # Train
        train_recall_weighed = mean(Model_.Epochs_Train_Acc)

        # Valid
        Y_pred = model_(X_valid).argmax(axis=1)
        Y = Y_valid
        valid_recall_weighed, valid_specificity_weighed, _ = recall_specificity_precision(Y,Y_pred,weighted_avg=True)
        valid_recall_simple, valid_specificity_simple, _ = recall_specificity_precision(Y,Y_pred,weighted_avg=False)

        # Test
        Y_pred = model_(X_test).argmax(axis=1)
        Y = Y_test
        test_recall_weighed, test_specificity_weighed, _ = recall_specificity_precision(Y,Y_pred,weighted_avg=True)
        test_recall_simple, test_specificity_simple, _ = recall_specificity_precision(Y,Y_pred,weighted_avg=False)


        if train_recall_weighed > Max_train_Acc: Max_train_Acc = train_recall_weighed
        if valid_recall_weighed > Max_valid_Acc: Max_valid_Acc = valid_recall_weighed
        if test_recall_weighed > Max_test_Acc: Max_test_Acc = test_recall_weighed
        new_row = pd.Series({'train_recall_weighed':train_recall_weighed,'valid_recall_weighed':valid_recall_weighed,'test_recall_weighed':test_recall_weighed
                                      ,'valid_recall_simple':valid_recall_simple,'test_recall_simple':test_recall_simple
                                      ,'valid_specificity_weighed':valid_specificity_weighed,'test_specificity_weighed':test_specificity_weighed
                                      ,'valid_specificity_simple':valid_specificity_simple,'test_specificity_simple':test_specificity_simple}, name='')
        CV_Details = CV_Details.append(new_row)
        pbar1.set_description(f'Max_train_Acc: {Max_train_Acc:.1%}, Max_valid_Acc: {Max_valid_Acc:.1%}, Max_test_Acc: {Max_test_Acc:.1%} ')
        pbar1.set_postfix_str(f'Train_Acc: {train_recall_weighed:.1%} , Valide_Acc: {valid_recall_weighed:.1%} , Test_Acc: {test_recall_weighed:.1%}')


        pickle.dump(CV_Details, open(G_path + '/08_Saved_Models_Outpus/Cross_Valid_Results/CV_280_Binary_'+ dat_hour,'wb'))

In [ ]:
col_recall = ['test_recall_weighed','valid_recall_weighed']
model_recalls = CV_Details[col_recall]
col_specificity = ['test_specificity_weighed','valid_specificity_weighed']
model_specificity = CV_Details[col_specificity]

# **plot Test Specificity and Sensitivity box-plots**

In [ ]:
plt.subplots(figsize=(8,4))
sns.boxplot(data=model_recalls)
plt.title('Test performance of 100 Cross-Validation',fontsize = 18)
plt.xlabel("Classifiers", fontsize = 14)
plt.ylabel("Accuracy", fontsize = 14)
plt.xticks(fontsize=16, rotation=0)
plt.show()

plt.subplots(figsize=(8,4))
sns.boxplot(data=model_specificity)
plt.title('Valid performance of 100 Cross-Validation',fontsize = 18)
plt.xlabel("Classifiers", fontsize = 14)
plt.ylabel("Accuracy", fontsize = 14)
plt.xticks(fontsize=16, rotation=0)
plt.show()
